In [ ]:
# Initial imports
import pandas as pd
from pathlib import Path


In [ ]:
# Data loading
file_path = Path("crypto_data.csv")
crypto_df = pd.read_csv(file_path)
crypto_df


In [ ]:
crypto_df.shape


In [ ]:
#filter 'IsTrading column'  for True only

crypto_traded_df = crypto_df[crypto_df['IsTrading']==True]

# , inplace = True

# en_users_df = users_df[users_df['stem_key_flag']==True]

crypto_traded_df


In [ ]:
crypto_traded_df.shape


In [ ]:
crypto_traded_df.dtypes


In [ ]:
#drop 'IsTrading', 'Unnamed', 'CoinName' columns  ....  not needed

crypto_traded_df.drop(columns=["Unnamed: 0", "CoinName", "IsTrading"], inplace=True)
crypto_traded_df


In [ ]:
new_crypto_df = crypto_traded_df.dropna()
new_crypto_df


In [ ]:
# filter for total coins mined  ( TotalCoinsMined > 0 )

mined_crypto_df = new_crypto_df[new_crypto_df['TotalCoinsMined'] > 0]
mined_crypto_df


In [ ]:
# drop CoinName column... not needed    <<<<  done in earlier drop  

# mined_crypto_df.drop(columns=["CoinName"], inplace=True)
# mined_crypto_df     


In [ ]:
mined_crypto_dummies_df = pd.get_dummies(mined_crypto_df, columns=['Algorithm', 'ProofType'])
mined_crypto_dummies_df


In [ ]:
# mined_crypto_dummies_df = pd.get_dummies(mined_crypto_df)    creates dummies for TotalCoinSupply ??  do we need?  
# mined_crypto_dummies_df


# change w/ dummy variables  532rows x4col   >>>  532rows x98col

In [ ]:
#standardize dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
import numpy as np

X = mined_crypto_dummies_df

scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)
X_scaled.shape



In [ ]:
# dir(X)

## PCA

In [ ]:
### Dimensionality Reduction (PCA)

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Applying PCA to reduce dimensions from 4 to 2       20 (unsupervised) - 1 - 6

# Initialize PCA model
pca = PCA(n_components=0.90)

# Get principal components for the data.
X_pca = pca.fit_transform(X_scaled)

df_X_pca = pd.DataFrame(data=X_pca)
# , columns=["principal component 1", "principal component 2"]

df_X_pca


In [ ]:
df_X_pca.shape


#  change w/ PCA  532rows x98col  >>> 532rows x74col


## t-SNE  


In [ ]:
### t-SNE

from sklearn.manifold import TSNE

tsne = TSNE(learning_rate=500)
tsne_features = tsne.fit_transform(X_pca)
tsne_features


In [ ]:
# Check that dimensions have been reduced to two
tsne_features.shape


In [ ]:
# Plot the results
plt.scatter(tsne_features[:,0], tsne_features[:,1])

plt.show()


In [ ]:
# Plot the results
plt.scatter(tsne_features[:,0], tsne_features[:,1], c=tsne_features[:,1])

plt.show()



# Appears to be 3 - 5  clusters 



## k-Means

In [ ]:
# Predicting clusters with k=3

# Initialize the k-means model
model = KMeans(n_clusters=3, random_state=0)

# Fit the model
model.fit(tsne_features)

# Predict clusters
predictions = model.predict(tsne_features)

# Add the predicted class column to the dataframe
df_X_pca["class"] = model.labels_
df_X_pca.head()


In [ ]:
# Cluster Analysis with k-Means

# Finding the best value for k
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
#     km.fit(df_X_pca)
    km.fit(tsne_features)
    inertia.append(km.inertia_)

# Creating the Elbow Curve
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)

plt.plot(df_elbow['k'], df_elbow['inertia'])
plt.xticks(list(range(11)))
plt.title('Elbow Curve')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.show()


## Elbow appears to be at K=3  

## 

# Recommendation

**After performing preprocessing of dataset and dimension reduction by PCA and t-SNE, k-Means was utilized in cluster analysis to determine if cryptocurrencies can be clustered together. From my findings, I would say that the data suggest that cryptocurrency can be clustered, however, further analysis would need to be done to be more conclusive.**   